In [1]:
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect

import numpy as np
from numpy.random import default_rng

import thztools as thz

In [2]:
n = 256  # Number of samples
m = 50  # Number of traces
ts = 0.05  # Sampling time [ps]
t0 = 2.5  # Peak pulse time [ps]
fs = 1 / ts  # Sampling frequency [THz]

sigma_alpha = 1e-4  # Additive noise amplitude [signal units]
sigma_beta = 1e-2  # Multiplicative noise amplitude [dimensionless]
sigma_tau = 1e-3  # Time base noise amplitude [ps]
sigma_parms = np.array([sigma_alpha, sigma_beta, sigma_tau])

seed = 0  # RNG seed
rng = default_rng(seed)

In [3]:
mu, t = thz.thzgen(n, ts, t0)
sigma = thz.noiseamp(sigma_parms, mu, ts)

# Use broadcasting to generate multiple simulated noisy measurements of y
x = mu + sigma * rng.standard_normal((m, n))

In [7]:
# tdnoisefit expects a column-oriented data array
a0 = np.ones(m)
eta0 = np.zeros(m)
result = thz.tdnoisefit(
    x.T, a0=a0, eta0=eta0, ts=ts, fix_a=False, fix_eta=False
)

In [8]:
p_names = ["var_alpha", "var_beta", "var_tau"]
for i in range(3):
    val = result[0]['var'][i] * (m / (m - 1))
    err = result[2]['err']['var'][i, i] * (m / (m - 1))
    print(f"Estimated {p_names[i]}: {val:.4g} ± {err:.4g}")

Estimated var_alpha: 9.961e-09 ± 1.505e-10
Estimated var_beta: 9.437e-05 ± 3.555e-06
Estimated var_tau: 8.819e-07 ± 7.455e-08


In [9]:
p_names = ["sigma_alpha", "sigma_beta", "sigma_tau"]
for i in range(3):
    val = np.sqrt(result[0]['var'][i] * (m / (m - 1)))
    err = 0.5 * result[2]['err']['var'][i, i] * (m / (m - 1)) / val
    print(f"Estimated {p_names[i]}: {val:.4g} ± {err:.4g}")
for i in range(3):
    val = np.sqrt(result[0]['var'][i] * (m / (m - 1))) / sigma_parms[i]
    err = (
        0.5
        * result[2]['err']['var'][i, i]
        * (m / (m - 1))
        / (sigma_parms[i] ** 2)
    )
    print(
        f"Estimated {p_names[i]}, relative to true value: {val:.3f} ± {err:.3f}"
    )

Estimated sigma_alpha: 9.981e-05 ± 7.541e-07
Estimated sigma_beta: 0.009715 ± 0.000183
Estimated sigma_tau: 0.0009391 ± 3.97e-05
Estimated sigma_alpha, relative to true value: 0.998 ± 0.008
Estimated sigma_beta, relative to true value: 0.971 ± 0.018
Estimated sigma_tau, relative to true value: 0.939 ± 0.037
